<a href="https://colab.research.google.com/github/olegoid/MLDS_ML_2022/blob/main/Hometask5_RecSys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Введение

В этом задании Вы продолжите работать с данными из семинара [Articles Sharing and Reading from CI&T Deskdrop](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop).

# Загрузка и предобработка данных

In [ ]:
import math

import numpy as np
import pandas as pd

from sklearn.metrics import ndcg_score

from scipy import sparse
from pandas.api.types import CategoricalDtype
from lightfm.cross_validation import random_train_test_split

from lightfm.data import Dataset

!pip install pymorphy2
import re
from pymorphy2 import MorphAnalyzer
from functools import lru_cache
from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')

from multiprocessing import Pool
from tqdm import tqdm

import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold

from sklearn.utils import column_or_1d
from sklearn.utils.multiclass import type_of_target
from sklearn.metrics import dcg_score

from tqdm import tqdm_notebook

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"olegdemchenko","key":"13af129dfdd9541e08105d7d4960060d"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

Загрузим данные и проведем предобраотку данных как на семинаре.

In [ ]:
!kaggle datasets download -d gspmoreira/articles-sharing-reading-from-cit-deskdrop
!unzip articles-sharing-reading-from-cit-deskdrop.zip -d articles

  0% 0.00/8.20M [00:00<?, ?B/s]
100% 8.20M/8.20M [00:00<00:00, 89.5MB/s]
Archive:  articles-sharing-reading-from-cit-deskdrop.zip
  inflating: articles/shared_articles.csv  
  inflating: articles/users_interactions.csv  


In [ ]:
articles_df = pd.read_csv("articles/shared_articles.csv")
articles_df = articles_df[articles_df["eventType"] == "CONTENT SHARED"]
print(articles_df.shape)
articles_df.head(2)

(3047, 13)


,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en


In [ ]:
interactions_df = pd.read_csv("articles/users_interactions.csv")
interactions_df.head(2)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US


In [ ]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [ ]:
# зададим словарь определяющий силу взаимодействия
event_type_strength = {
    "VIEW": 1.0,
    "LIKE": 2.0,
    "BOOKMARK": 2.5,
    "FOLLOW": 3.0,
    "COMMENT CREATED": 4.0,
}

interactions_df["eventStrength"] = interactions_df.eventType.apply(
    lambda x: event_type_strength[x]
)

Оставляем только тех пользователей, которые произамодействовали более чем с пятью статьями.

In [ ]:
users_interactions_count_df = (
    interactions_df.groupby(["personId", "contentId"])
    .first()
    .reset_index()
    .groupby("personId")
    .size()
)
print("# users:", len(users_interactions_count_df))

users_with_enough_interactions_df = users_interactions_count_df[
    users_interactions_count_df >= 5
].reset_index()[["personId"]]
print("# users with at least 5 interactions:", len(users_with_enough_interactions_df))

# users: 1895
# users with at least 5 interactions: 1140


Оставляем только те взаимодействия, которые относятся к отфильтрованным пользователям.

In [ ]:
interactions_from_selected_users_df = interactions_df.loc[
    np.in1d(interactions_df.personId, users_with_enough_interactions_df)
]

In [ ]:
print(f"# interactions before: {interactions_df.shape}")
print(f"# interactions after: {interactions_from_selected_users_df.shape}")

# interactions before: (72312, 9)
# interactions after: (69868, 9)


Объединяем все взаимодействия пользователя по каждой статье и сглаживаем полученный результат, взяв от него логарифм.

In [ ]:
def smooth_user_preference(x):
    return math.log(1 + x, 2)


interactions_full_df = (
    interactions_from_selected_users_df.groupby(["personId", "contentId"])
    .eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index()
    .set_index(["personId", "contentId"])
)
interactions_full_df["last_timestamp"] = interactions_from_selected_users_df.groupby(
    ["personId", "contentId"]
)["timestamp"].last()

interactions_full_df = interactions_full_df.reset_index()
interactions_full_df.head(5)

,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.000000,1470395911
1,-1007001694607905623,-6623581327558800021,1.000000,1487240080
2,-1007001694607905623,-793729620925729327,1.000000,1472834892
3,-1007001694607905623,1469580151036142903,1.000000,1487240062
4,-1007001694607905623,7270966256391553686,1.584963,1485994324


Разобьём выборку на обучение и контроль по времени.

In [ ]:
from sklearn.model_selection import train_test_split

split_ts = 1475519530
interactions_train_df = interactions_full_df.loc[
    interactions_full_df.last_timestamp < split_ts
].copy()
interactions_test_df = interactions_full_df.loc[
    interactions_full_df.last_timestamp >= split_ts
].copy()

print(f"# interactions on Train set: {len(interactions_train_df)}")
print(f"# interactions on Test set: {len(interactions_test_df)}")

interactions_train_df

# interactions on Train set: 29329
# interactions on Test set: 9777


,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.0,1470395911
2,-1007001694607905623,-793729620925729327,1.0,1472834892
6,-1032019229384696495,-1006791494035379303,1.0,1469129122
7,-1032019229384696495,-1039912738963181810,1.0,1459376415
8,-1032019229384696495,-1081723567492738167,2.0,1464054093
...,...,...,...,...
39099,997469202936578234,9112765177685685246,2.0,1472479493
39100,998688566268269815,-1255189867397298842,1.0,1474567164
39101,998688566268269815,-401664538366009049,1.0,1474567449
39103,998688566268269815,6881796783400625893,1.0,1474567675


Для удобства подсчёта качества запишем данные в формате, где строка соответствует пользователю, а столбцы будут истинными метками и предсказаниями в виде списков.

In [ ]:
interactions = (
    interactions_train_df.groupby("personId")["contentId"]
    .agg(lambda x: list(x))
    .reset_index()
    .rename(columns={"contentId": "true_train"})
    .set_index("personId")
)

interactions["true_test"] = interactions_test_df.groupby("personId")["contentId"].agg(
    lambda x: list(x)
)

# заполнение пропусков пустыми списками
interactions.loc[pd.isnull(interactions.true_test), "true_test"] = [
    ""
    for x in range(
        len(interactions.loc[pd.isnull(interactions.true_test), "true_test"])
    )
]

interactions.head(1)

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."


# Библиотека LightFM

Для рекомендации Вы будете пользоваться библиотекой [LightFM](https://making.lyst.com/lightfm/docs/home.html), в которой реализованы популярные алгоритмы. Для оценивания качества рекомендации, как и на семинаре, будем пользоваться метрикой *precision@10*.

In [ ]:
!pip install lightfm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 KB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp39-cp39-linux_x86_64.whl size=889509 sha256=e8a3d218456712f23da07407ee51ea5322c0bf5497993a0e15ad983dbb854558
  Stored in directory: /root/.cache/pip/wheels/d8/65/93/6ac8180274dc2e8f86ff326be62da1dfa55dc158fd45faba7d
Successfully built lightfm


In [ ]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

## Задание 1 (1.5 балла)

Модели в LightFM работают с разреженными матрицами. Создайте разреженные матрицы `data_train` и `data_test` (размером количество пользователей на количество статей), такие что на пересечении строки пользователя и столбца статьи стоит сила их взаимодействия, если взаимодействие было, и стоит ноль, если взаимодействия не было.

In [ ]:
def convert_to_sparse_matrix(df):
    unique_person_id = interactions_full_df['personId'].unique()
    unique_content_id = interactions_full_df['contentId'].unique()

    shape = (len(unique_person_id), len(unique_content_id))
    
    person_cat = CategoricalDtype(categories=sorted(unique_person_id), ordered=True)
    content_cat = CategoricalDtype(categories=sorted(unique_content_id), ordered=True)
    
    person_index = df["personId"].astype(person_cat).cat.codes
    content_index = df["contentId"].astype(content_cat).cat.codes
    
    coo = sparse.coo_matrix((df["eventStrength"], (person_index, content_index)), shape=shape)
    csr = coo.tocsr()
    
    return csr

data_train = convert_to_sparse_matrix(interactions_train_df)
print(data_train.shape)

data_test = convert_to_sparse_matrix(interactions_test_df)
print(data_test.shape)

(1140, 2984)
(1140, 2984)


## Задание 2 (0.5 балла)

Обучите модель LightFM с `loss="warp"` и посчитайте *precision@10* на тесте.

In [ ]:
RANDOM_STATE = 42

model = LightFM(loss='warp', random_state=RANDOM_STATE)
model = model.fit(data_train, epochs=100)

In [ ]:
test_precision_at_10 = precision_at_k(model, data_test, train_interactions = data_train, k=10).mean()
print("Test precision@10: " + str(test_precision_at_10))

Test precision@10: 0.007433809


## Задание 3 (2 балла)

При вызове метода `fit` LightFM позволяет передавать в `item_features` признаковое описание объектов. Воспользуемся этим. Будем получать признаковое описание из текста статьи в виде [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF) (можно воспользоваться `TfidfVectorizer` из scikit-learn). Создайте матрицу `feat` размером количесвто статей на размер признакового описание и обучите LightFM с `loss="warp"` и посчитайте precision@10 на тесте.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
feat = vectorizer.fit_transform(articles_df['text'])

model = LightFM(loss='warp', random_state=RANDOM_STATE)
model = model.fit(data_train, item_features=feat, epochs=100)

test_precision_at_10 = precision_at_k(model, data_test, item_features=feat, train_interactions=data_train, k=10).mean()
print("Test precision@10: " + str(test_precision_at_10))

Test precision@10: 0.007841141


## Задание 4 (1.5 балла)

В задании 3 мы использовали сырой текст статей. В этом задании необходимо сначала сделать предобработку текста (привести к нижнему регистру, убрать стоп слова, привести слова к номральной форме и т.д.), после чего обучите модель и оценить качество на тестовых данных.

In [ ]:
m = MorphAnalyzer()
regex = re.compile("[A-z]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]


mystopwords = stopwords.words('english') 
def remove_stopwords(lemmas, stopwords = mystopwords):
    return [w for w in lemmas if not w in stopwords and len(w) > 3]

def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)
    
    return ' '.join(remove_stopwords(lemmas))

In [ ]:
with Pool(4) as p:
    lemmas = list(tqdm(p.imap(clean_text, articles_df['text']), total=len(articles_df)))
    
articles_df['lemmas'] = lemmas
articles_df.sample(5)

100%|██████████| 3047/3047 [01:06<00:00, 45.90it/s]


,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang,lemmas
387,1460565739,CONTENT SHARED,-2181071289972835602,-1443636648652872475,-7298018010644449122,NaN,NaN,NaN,HTML,http://www.networkworld.com/article/3055195/cl...,Google cloud to OpenStack users: Come on in!,Open source cloud computing platform OpenStack...,en,open source cloud computing platform openstack...
757,1462300131,CONTENT SHARED,7078276984547732873,-1443636648652872475,5914495757799096398,NaN,NaN,NaN,HTML,https://hbr.org/2016/05/how-companies-are-usin...,How Companies Are Using Machine Learning to Ge...,Create a FREE account to: Get eight free artic...,en,create free account eight free articles month ...
1837,1469129118,CONTENT SHARED,-1006791494035379303,-1032019229384696495,729690485230072651,NaN,NaN,NaN,HTML,https://techcrunch.com/2016/07/20/google-unlea...,Google unleashes DeepMind on energy-hungry dat...,DeepMind may be a master at one of the most co...,en,deepmind master complex games earth handle ene...
409,1460594772,CONTENT SHARED,4555339116751579515,3891637997717104548,3664397758473282254,NaN,NaN,NaN,HTML,http://fourkitchens.com/blog/article/meet-aqui...,Meet Aquifer: A build system for easier Drupal...,What is a build system? Build systems simplify...,en,build system build systems simplify developmen...
2666,1477590006,CONTENT SHARED,-6564402465035266866,-8845298781299428018,-326193745783508787,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1...,SP,BR,HTML,https://techcrunch.com/2016/10/27/twitter-is-s...,Twitter is shutting down Vine,With Twitter's future still in question follow...,en,twitter future still question following failed...


In [ ]:
vectorizer = TfidfVectorizer()
feat = vectorizer.fit_transform(articles_df['lemmas'])

model = LightFM(loss='warp', random_state=RANDOM_STATE)
model = model.fit(data_train, item_features=feat, epochs=100)

test_precision_at_10 = precision_at_k(model, data_test, item_features=feat, train_interactions=data_train, k=10).mean()
print("Test precision@10: " + str(test_precision_at_10))

Test precision@10: 0.007942974


Улучшилось ли качество предсказания?

Да, но разница незначительна.

## Задание 5 (1.5 балла)

Подберите гиперпараметры модели LightFM (`n_components` и др.) для улучшения качества модели.

In [ ]:
model = LightFM(loss='warp', random_state=RANDOM_STATE)

randint = stats.randint(low=1, high=65)
randint.random_state = RANDOM_STATE

gamma = stats.gamma(a=1.2, loc=0, scale=0.13)
gamma.random_state = RANDOM_STATE

parameters = {
  "no_components": randint,
  "learning_schedule": ["adagrad", "adadelta"],
  "learning_rate": gamma,
  "item_alpha": (1e-8, 1e-6),
  "user_alpha": (1e-8, 1e-6),
  "max_sampled": (5, 15),
}

def scorer(est, x, y=None):
    score = precision_at_k(est, x, k=10).mean()
    return score

class CV(KFold):
      def split(self, X, y=None, groups=None):
          idx = np.arange(X.shape[0])
          for _ in range(self.n_splits):
              yield idx, idx

cv = CV(n_splits=3)

search = RandomizedSearchCV(estimator=model,
                            param_distributions=parameters,
                            n_iter=100,
                            scoring=scorer,
                            random_state=RANDOM_STATE,
                            cv=cv)
search.fit(data_train)
print(search.best_params_)

{'item_alpha': 1e-06, 'learning_rate': 0.13970472792796088, 'learning_schedule': 'adagrad', 'max_sampled': 15, 'no_components': 58, 'user_alpha': 1e-08}


## Задание 6 (1 балл)

Реализуйте функции для вычисления следующих метрик:
* precision@k
* recall@k
* NDCG@k



In [ ]:
# precision@k
def precision_at_k_custom(predictions, k):
    predictions.data = np.less(predictions.data, k, predictions.data)
    precision = np.squeeze(np.array(predictions.sum(axis=1)))

    return (precision / k)

# recall@k
def recall_at_k_custom(predictions, test, k):
    predictions.data = np.less(predictions.data, k, predictions.data)

    correct_predictions = np.squeeze(np.array(predictions.sum(axis=1)))
    denominator = np.squeeze(test.getnnz(axis=1))

    # doing this to avoid division by zero
    correct_predictions = correct_predictions[test.getnnz(axis=1) > 0]
    denominator = denominator[test.getnnz(axis=1) > 0]

    return (correct_predictions / denominator)

# NDCG@k
def ndcg_score_custom(y_true, y_score, k):
    best = dcg_score(y_true, y_true, k = k)
    actual = dcg_score(y_true, y_score, k = k)
    return actual / best

def dcg_score(y_true, y_score, k):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])

    gains = 2 ** y_true - 1

    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)


## Задание 7 (1 балл)

Вычислите значения реализованных метрик для $k=10$ для лучшей полученной модели в предыдущих шагах.

Найдите уже реализованные варианты этих метрик в библиотеках lightfm и sklearn. Сравните полученные у вас значения метрик с результатами встроенных в библиотеки метрик.

In [ ]:
model = LightFM(loss='warp', random_state=RANDOM_STATE)
model = model.fit(data_train, item_features=feat, epochs=100)

Custom precision@10: [0. 0. 0. ... 0. 0. 0.]
Library precision@10: 0.007942974


In [ ]:
predictions = model.predict_rank(data_test, train_interactions=data_train, item_features=feat, num_threads=5, check_intersections=True)

custom_precision = precision_at_k_custom(predictions, 10).mean()
print("Custom precision@10: " + str(custom_precision))

library_precision = precision_at_k(model, data_test, item_features=feat, train_interactions=data_train, k=10).mean()
print("Library precision@10: " + str(library_precision))

Custom precision@10: 0.0068421056
Library precision@10: 0.007942974


In [ ]:
predictions = model.predict_rank(data_test, train_interactions=data_train, item_features=feat, num_threads=5, check_intersections=True)

custom_recall = recall_at_k_custom(predictions, data_test, 10).mean()
print("Custom recall@10: " + str(custom_recall))

library_recall = recall_at_k(model, data_test, item_features=feat, train_interactions=data_train, k=10).mean()
print("Library recall@10: " + str(library_recall))

Custom recall@10: 0.010447595355663896
Library recall@10: 0.010447595355663896


In [ ]:
model = model.fit(data_train, item_features=feat, epochs=100)
model.fit(data_train)

# transform the data to be passed to ndcg
person_id = np.asarray(
    [u for u in range(interactions_full_df['personId'].nunique())])

content_id = np.asarray(
    [i for i in range(interactions_full_df['contentId'].nunique())])

pred = [model.predict(int(i), content_id) for i in person_id]

library_ndcg = ndcg_score(data_test.toarray(), pred, k = 10)
print("Library ndcg@10: " + str(library_ndcg))

custom_ndcg = ndcg_score_custom(data_test.toarray(), pred, 10)
print("Custom ndcg@10: " + str(custom_ndcg))

Library ndcg@10: 0.0037707189442508122
Custom ndcg@10: 0.0060065691235556004


Результаты оказались довольно хорошими. Библиотечная precision_at_k и precision_at_k_custom имеют схожие значения. Recall имеет идентичный результат.

NDCG различаются весьма существенно. Скор библиотеки имеет множество параметров, которые определенно влияют на итоговую оценку. Моя реализация довольно проста, dsg экспоненциальна.

## Задание 8 (1 балл)

Реализуйте алгоритм ALS и примените его для решения задачи ноутбука.

**ALS**

Итак, поставлена задача построения модели со скрытыми переменными (latent factor model) для коллаборативной фильтрации:

$$ \sum_{u,i} (r_{ui} - \langle p_u, q_i \rangle)^2 \to \min_{P,Q}$$

Суммирование ведется по всем парам $(u, i),$ для которых известен рейтинг $r_{ui}$ (и только по ним), а $p_u, q_i$ – латентные представления пользователя~$u$ и товара $i$, соответственно, матрицы $P, Q$ получаются путем записывания по столбцам векторов $p_u, q_i$ соответственно.

Подход ALS (Alternating Least Squares) решает задачу, попеременно фиксируя матрицы $P$ и $Q$, — оказывается, что, зафиксировав одну из матриц, можно выписать аналитическое решение задачи для другой.

$$\nabla_{p_u} \bigg[ \sum_{u,i} (r_{ui} - \langle p_u, q_i \rangle)^2 \bigg] = \sum_{i} 2(r_{ui} - \langle p_u, q_i \rangle)q_i = 0$$

Воспользовавшись тем, что $a^Tbc = cb^Ta$, получим
$$\sum_{i} r_{ui}q_i - \sum_i q_i q_i^T p_u = 0.$$

Тогда окончательно каждый столбец матрицы $P$ можно найти по формуле
$$p_u = \bigg( \sum_i q_i q_i^T\bigg)^{-1}\sum_ir_{ui}q_i \;\; \forall u,$$

аналогично для столбцов матрицы $Q$
$$q_i = \bigg( \sum_u p_u p_u^T\bigg)^{-1}\sum_ur_{ui}p_u \;\; \forall i.$$

Таким образом мы можем решать оптимизационную задачу, поочередно фиксируя одну из матриц $P$ или $Q$ и проводя оптимизацию по второй.

**Оригинальная статья c постановкой задачи для ALS на explicit feedback:**

* Bell, R.M. and Koren, Y., 2007, October. Scalable collaborative filtering with jointly derived neighborhood interpolation weights. In Seventh IEEE international conference on data mining (ICDM 2007) (pp. 43-52). IEEE.

**Оригинальная статья с ALS для implicit данных, которая стала более известной:**

* Hu, Y., Koren, Y. and Volinsky, C., 2008, December. Collaborative filtering for implicit feedback datasets. In 2008 Eighth IEEE international conference on data mining (pp. 263-272). Ieee.


In [ ]:
from scipy.linalg import svd

ratings = pd.pivot_table(
    interactions_train_df,
    values='eventStrength',
    index='personId',
    columns='contentId').fillna(0)

U, sigma, V = svd(ratings)
print(ratings.shape, U.shape, sigma.shape, V.shape)

(1112, 2366) (1112, 1112) (1112,) (2366, 2366)


In [ ]:
Sigma = np.zeros((1112, 2366))
Sigma[:1112, :1112] = np.diag(sigma)

new_ratings = U.dot(Sigma).dot(V)

print(sum(sum((new_ratings - ratings.values) ** 2)))

9.51524958314499e-25


In [ ]:
K = 100

sigma[K:] = 0
Sigma = np.zeros((1112, 2366))
Sigma[:1112, :1112] = np.diag(sigma)

new_ratings = U.dot(Sigma).dot(V)

print(sum(sum((new_ratings - ratings.values) ** 2)))
print(sum(sum((ratings.values.mean() - ratings.values) ** 2)))

25843.42489891279
78622.13000466184


In [ ]:
new_ratings = pd.DataFrame(new_ratings, index=ratings.index, columns=ratings.columns)

top_k = 10
predictions = []

for personId in tqdm_notebook(interactions.index):
    prediction = (
        new_ratings
        .loc[personId]
        .sort_values(ascending=False)
        .index.values
    )
    
    predictions.append(
        list(prediction[~np.in1d(
            prediction,
            interactions.loc[personId, 'true_train'])])[:top_k])

interactions['prediction_svd'] = predictions

<ipython-input-130-b4e4b8b95f81>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for personId in tqdm_notebook(interactions.index):


  0%|          | 0/1112 [00:00<?, ?it/s]

In [ ]:
def calc_precision(column):
    return (
        interactions
        .apply(
            lambda row:
            len(set(row['true_test']).intersection(
                set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),
            axis=1)).mean()
            
calc_precision('prediction_svd')

0.012212989310270756